In [157]:
import pandas as pd
import numpy as np

#get datasets
people = pd.read_csv('../../baseball data/core/People.csv')
appearance = pd.read_csv('../../baseball data/core/Appearances.csv')
hall_of_fame = pd.read_csv('../../baseball data/contrib/HallOfFame.csv')
batting = pd.read_csv('../../baseball data/core/Batting.csv')
pitching = pd.read_csv('../../baseball data/core/Pitching.csv')
fielding = pd.read_csv('../../baseball data/core/Fielding.csv')
teams = pd.read_csv('../../baseball data/core/Teams.csv')
salary = pd.read_csv('../../baseball data/contrib/Salaries.csv')

juniors = ["guerrvl02", "tatisfe02", "penato02", "borbope02", "cruzjo02",\
            "dejesiv02", "alomasa02", "johnsra02", "spraged02", "gwynnto02",\
            "bruckea02", "beamoch02", "matthga02", "walshed02", "leitema02",\
            "mcculla02", "smithdw02", "hairsje02", "schuljo05", "sullibi04",\
            "deshide02", "ripkeca01", "lombast02", "maybejo02", "younger03",\
            "rosepe02", "averiea02", "bagbyji02", "liebhgl02", "troskha02",\
            "griffke02", "collied02", "krausle02", "amaroru02", "nichoch02",\
            "raineti02", "rathfr02", "stottme02", "armasto02", "bradlja02",\
            "acunaro01", "gurrilo01"]

In [159]:
def jr(nm, id):
    if id in juniors:
        return nm + ' Jr.'
    else:
        return nm

def gen(x):
    if 1700 <= x <= 1882:
        return 'Pre-Generational'
    elif 1883 <= x <= 1900:
        return 'Lost Generation'
    elif 1901 <= x <= 1927:
        return 'Greatest Generation'
    elif 1928 <= x <= 1945:
        return 'Silent Generation'
    elif 1946 <= x <= 1964:
        return 'Baby Boomers'
    elif 1965 <= x <= 1980:
        return 'Gen X'
    elif 1981 <= x <= 1996:
        return 'Millenial'
    elif 1997 <= x <= 2012:
        return 'Gen Z'
    elif 2013 <= x <= 2025:
        return 'Gen Alpha'
    else:
        return 'Unknown'

pd_df = people[~people['playerID'].str.endswith('99')].\
    drop(['deathYear', 'deathMonth', 'deathDay', 'deathCountry', 'deathCity', 'deathState'], axis=1)
    
#fix player birth location fuckery
pd_df.loc[:,'birthCountry'] = pd_df['birthCountry'].apply(lambda x: x if x != '' else 'Unknown')
pd_df.loc[:,'birthState'] = pd_df['birthState'].apply(lambda x: x if x != '' else 'Unknown')
pd_df.loc[:,'birthCity'] = pd_df['birthCity'].apply(lambda x: x if x != '' else 'Unknown')
pd_df.loc[:,'birthCity'] = pd_df['birthCity'] + ', ' + pd_df['birthState'] 

#add jr to the juniors last name
pd_df.loc[:,'nameLast']  = pd_df[['nameLast', 'playerID']].apply(lambda x: jr(x['nameLast'], x['playerID']), axis=1)

#fix player name fuckery
pd_df.loc[:,'nameFirst'] = pd_df['nameFirst'].apply(lambda x: x if x != '' else 'None Given')
pd_df['fullName'] = pd_df['nameFirst']+' '+pd_df['nameLast']

#fix player birth date fuckery
pd_df.loc[:,'birthYear'] = pd_df['birthYear'].fillna(0).apply(lambda x: x if x != 0 else 1820)
pd_df.loc[:,'birthMonth'] = pd_df['birthMonth'].fillna(0).apply(lambda x: x if x != 0 else 1)
pd_df.loc[:,'birthDay'] = pd_df['birthDay'].fillna(0).apply(lambda x: x if x != 0 else 1)
pd_df['birthDate'] = pd_df.apply(lambda x: "{}-{}-{}".format(int(x['birthYear']), int(x['birthMonth']), int(x['birthDay'])), axis=1)
pd_df['birthDate'] = pd.to_datetime(pd_df['birthDate'])

#determine player generation
pd_df['generation'] = pd_df['birthYear'].apply(lambda x: gen(x))

#fix missing bat and hit handedness
pd_df['bats'].fillna('U', inplace=True)
pd_df['throws'].fillna('U', inplace=True)

#copy debut to debutDate because that's the way the front end needs it
pd_df['debutDate'] = pd_df['debut']



In [ ]:
#get player's primary team
team_app = appearance[['playerID', 'teamID', 'G_all']].groupby(['playerID', 'teamID'], as_index=False).sum()
team_app = team_app.sort_values('G_all', ascending=False).drop_duplicates(['playerID'])

In [ ]:
#determine hof status
hof = hall_of_fame[(hall_of_fame['inducted'] == 'Y') | (hall_of_fame['inducted'] == 'y')][['playerID', 'inducted']]

In [ ]:
#determine player primary position
def position(x):
    
    if (x.p1 == 'G_p'):
        
        if x.G_p == 0:
            p = 'PH'
        
        elif x.GS/x.G_p < .25:
            p = 'RP'
        
        else:
            p = 'SP'
    
    else: 
        p = x['p1'].replace('G_', '').upper()
    
    return p
  
app_agg = appearance.drop(['yearID', 'teamID', 'lgID', 'G_all', 'G_batting', 'G_defense'], axis=1).groupby(['playerID']).sum()
app_agg['p1'] = app_agg.drop(['GS'], axis=1).idxmax(axis=1)
app_agg['position'] = app_agg.apply(position, axis=1)


In [ ]:
p_df = pitching[['playerID', 'IPouts']].groupby('playerID').agg(PIPO = ('IPouts', 'sum'))
f_df = fielding[['playerID', 'InnOuts']].groupby('playerID').agg(FIPO = ('InnOuts', 'sum'))

b_df = batting[['playerID', 'AB', 'BB', 'HBP', 'SF', 'SH']].groupby('playerID', as_index=False).sum()
b_df['PA'] = b_df.drop(['playerID'], axis=1).sum(axis=1)

#join to a single dataframe
playtime = b_df[['playerID', 'PA']].merge(p_df, how='outer', on='playerID').merge(f_df, how='outer', on='playerID').fillna(0)


In [ ]:
#bring them all together

player_details = pd_df[['playerID', 'birthCountry', 'birthState', 'birthCity', 'birthDate', 'nameFirst', 'nameLast', 'fullName',\
                        'weight', 'height', 'bats', 'throws', 'debutDate', 'finalGame', 'retroID', 'bbrefID']].\
                        merge(hof, how='left').\
                        merge()